# Correlating Returns

In [14]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load API Keys from Environment Variables

In [15]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

## Get AAPL Returns for Past Month

In [16]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp("2020-08-30", tz="America/New_York").isoformat()
past_date = pd.Timestamp("2020-08-26", tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=past_date,
    end=current_date,
    after=None,
    until=None,
).df

# Display data
df.head()

AAPL                                  
                               open    high     low   close    volume
2020-08-26 00:00:00-04:00  504.7165  507.97  500.33  506.19  37949023
2020-08-27 00:00:00-04:00  508.5700  509.94  495.33  500.04  36378278
2020-08-28 00:00:00-04:00  504.0500  505.77  498.31  498.83  38207206

In [17]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df = df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()

,close
2020-08-26,506.19
2020-08-27,500.04
2020-08-28,498.83


In [18]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change().dropna()

# Display sample data
aapl_returns.head()

,close
2020-08-27,-0.01215
2020-08-28,-0.00242


In [19]:
date = datetime.strptime(current_date[:10], "%Y-%m-%d")

In [20]:
str(date)

'2020-08-30 00:00:00'

In [21]:
current_date[:10]

'2020-08-30'

In [22]:
articles = newsapi.get_everything(
            q='apple',
            from_param='2020-08-26',
#             to=str(date),
            language="en",
            sort_by="relevancy",
            page=1,
        )

In [23]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")

    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=date.strftime('%Y-%m-%d'),
            to=date.strftime('%Y-%m-%d'),
            language="en",
            sort_by="relevancy",
            page=1,
        )            
        headlines = [article['title'] for article in articles['articles']]
        
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

In [24]:
# Get first topic
aapl_headlines, dates = get_headlines("apple")

Fetching news about 'apple'
******************************
retrieving news from: 2020-08-30 00:00:00
retrieving news from: 2020-08-29 00:00:00
retrieving news from: 2020-08-28 00:00:00
retrieving news from: 2020-08-27 00:00:00


In [25]:
# Get second topic
trade_headlines, _ = get_headlines("trade")

Fetching news about 'trade'
******************************
retrieving news from: 2020-08-30 00:00:00
retrieving news from: 2020-08-29 00:00:00
retrieving news from: 2020-08-28 00:00:00
retrieving news from: 2020-08-27 00:00:00


In [26]:
# Get third topic
economy_headlines, _ = get_headlines("economy")

Fetching news about 'economy'
******************************
retrieving news from: 2020-08-30 00:00:00
retrieving news from: 2020-08-29 00:00:00
retrieving news from: 2020-08-28 00:00:00
retrieving news from: 2020-08-27 00:00:00


In [27]:
# Get fourth topic
iphone_headlines, _ = get_headlines("iphone")

Fetching news about 'iphone'
******************************
retrieving news from: 2020-08-30 00:00:00
retrieving news from: 2020-08-29 00:00:00
retrieving news from: 2020-08-28 00:00:00
retrieving news from: 2020-08-27 00:00:00


In [28]:
# Get fifth topic
gold_headlines, _ = get_headlines("gold")

Fetching news about 'gold'
******************************
retrieving news from: 2020-08-30 00:00:00
retrieving news from: 2020-08-29 00:00:00
retrieving news from: 2020-08-28 00:00:00
retrieving news from: 2020-08-27 00:00:00


In [29]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [30]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [31]:
# Get averages of each topics sentiment
aapl_avg = headline_sentiment_summarizer_avg(aapl_headlines)
trade_avg = headline_sentiment_summarizer_avg(trade_headlines)
economy_avg = headline_sentiment_summarizer_avg(economy_headlines)
iphone_avg = headline_sentiment_summarizer_avg(iphone_headlines)
gold_avg = headline_sentiment_summarizer_avg(gold_headlines)

In [32]:
# Combine Sentiment Averages into DataFrame
topic_sentiments = pd.DataFrame(
    {
        "aapl_avg": aapl_avg,
        "trade_avg": trade_avg,
        "economy_avg": economy_avg,
        "iphone_avg": iphone_avg,
        "gold_avg": gold_avg,
    }
)

In [33]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)

In [34]:
# Merge with AAPL returns
topic_sentiments = aapl_returns.join(topic_sentiments).dropna(how="any")

# Display data
display(topic_sentiments)

,close,aapl_avg,trade_avg,economy_avg,iphone_avg,gold_avg
2020-08-27,-0.01215,-0.027505,-0.029595,-0.144850,0.07766,0.165215
2020-08-28,-0.00242,-0.083950,-0.007715,0.095855,-0.04384,-0.015305


In [35]:
# Correlate the headlines' sentiment to returns
topic_sentiments.corr().style.background_gradient()

,close,aapl_avg,trade_avg,economy_avg,iphone_avg,gold_avg
close,1.000000,-1.000000,1.000000,1.000000,-1.000000,-1.000000
aapl_avg,-1.000000,1.000000,-1.000000,-1.000000,1.000000,1.000000
trade_avg,1.000000,-1.000000,1.000000,1.000000,-1.000000,-1.000000
economy_avg,1.000000,-1.000000,1.000000,1.000000,-1.000000,-1.000000
iphone_avg,-1.000000,1.000000,-1.000000,-1.000000,1.000000,1.000000
gold_avg,-1.000000,1.000000,-1.000000,-1.000000,1.000000,1.000000
